In [1]:
# from preql.compiler import compile
## set up imports of local module code
import os
nb_path = os.path.abspath("")
from sys import path
from os.path import dirname

path.insert(0,  dirname(nb_path))

# nb_path

from preql.core.models import Select, Grain
from preql.core.query_processor import process_query
from preql.parser import parse
from logging import StreamHandler, INFO
from preql.constants import logger
logger.setLevel(INFO)
logger.addHandler(StreamHandler())

TEST_SETUP = r"""


key order_key int;
property order_key.order_amount float;
metric total_order_amount <- sum(order_amount);

key key int;
property key.weight float;
property key.name string;


key sub_category_key int;
key category_key int;

property category_key.category_name string;


datasource product_info (
productKey:key,
productSubCategoryKey:sub_category_key,
englishProductName:name
)
grain (key)
address AdventureWorksDW2019.dbo.DimProduct
;

datasource orders (
order:order_key,
product_key:key,
order_amount:order_amount
)
grain(order_key)
address AdventureWorksDW2019.dbo.DimOrder;


datasource product_sub_category (
productSubCategoryKey:sub_category_key,
productCategoryKey:category_key,
)
grain (sub_category_key)
address AdventureWorksDW2019.dbo.DimProductSubCategory
;

datasource product_category (
productCategoryKey:category_key,
englishProductCategoryName:category_name
)
address AdventureWorksDW2019.dbo.DimProductCategory
;

select 
    category_name,
    total_order_amount;
"""


env, parsed = parse(TEST_SETUP)
select: Select = parsed[-1]

assert select.grain == Grain(components=[env.concepts["category_name"]])

process_query(statement=select, environment=env)


[QUERY BUILD] getting source datasource for query with output ['local.category_name<local.category_key>', 'local.total_order_amount<>']
[CONCEPT DETAIL] error with finding constant source: 'SelectNode' object has no attribute 'resolve_direct_select'
[CONCEPT DETAIL] Beginning sourcing loop for ['local.category_name<local.category_key>', 'local.total_order_amount<>']
	[CONCEPT DETAIL] error with finding constant source: 'SelectNode' object has no attribute 'resolve_direct_select'
	[CONCEPT DETAIL] Beginning sourcing loop for ['local.order_amount<local.order_key>', 'local.category_name<local.category_key>']
		[CONCEPT DETAIL - SELECT NODE] Attempting to resolve select to reach local.order_amount,local.category_name, have 1 optional concept, pass 1
		[CONCEPT DETAIL - SELECT NODE] Found joined source from [Address(location='AdventureWorksDW2019.dbo.DimOrder'), Address(location='AdventureWorksDW2019.dbo.DimProductCategory'), Address(location='AdventureWorksDW2019.dbo.DimProduct'), Address(

ProcessedQuery(output_columns=[Concept(name='category_name', datatype=<DataType.STRING: 'string'>, purpose=<Purpose.PROPERTY: 'property'>, metadata=Metadata(description=None, line_number=16, concept_source=<ConceptSource.MANUAL: 'manual'>), lineage=None, namespace='local', keys=[Concept(name='category_key', datatype=<DataType.INTEGER: 'int'>, purpose=<Purpose.KEY: 'key'>, metadata=Metadata(description=None, line_number=14, concept_source=<ConceptSource.MANUAL: 'manual'>), lineage=None, namespace='local', keys=None, grain=Grain(components=[Concept(name='category_key', datatype=<DataType.INTEGER: 'int'>, purpose=<Purpose.KEY: 'key'>, metadata=Metadata(description=None, line_number=None, concept_source=<ConceptSource.MANUAL: 'manual'>), lineage=None, namespace='local', keys=None, grain=Grain(components=[Concept(name='category_key', datatype=<DataType.INTEGER: 'int'>, purpose=<Purpose.KEY: 'key'>, metadata=Metadata(description=None, line_number=None, concept_source=<ConceptSource.MANUAL: '